In [1]:
from bs4 import BeautifulSoup as bs
import selenium
import requests 
import pandas as pd
import datetime

In [3]:
with open("spreads.html") as fp:
    soup = bs(fp, "html.parser")

In [10]:
spreads_results = pd.read_html("spreads.html")[0]

In [12]:
spreads_results.to_csv("spreads.csv")

In [2]:
from tda import auth, client
import json
 
token_path = R'.\token.pickle'
api_key = 'DRSMU4TL964FO3QNBQHVL78X9SUPGGIL@AMER.OAUTHAP'
redirect_uri = "http://localhost"
try:
    c = auth.client_from_token_file(token_path, api_key)
except FileNotFoundError:
    from selenium import webdriver
    with webdriver.Chrome() as driver:
        c = auth.client_from_login_flow(
            driver, api_key, redirect_uri, token_path)

In [4]:
para = c.Options

chains = c.get_option_chain(symbol="SPY", contract_type=para.ContractType.PUT, to_date=datetime.date(2021,5,28))

In [31]:
initial = pd.json_normalize(chains.json(), record_path="monthlyStrategyList")

In [6]:
initial = pd.json_normalize(chains.json())

In [7]:
initial

,symbol,status,underlying,strategy,interval,isDelayed,isIndex,interestRate,underlyingPrice,volatility,...,putExpDateMap.2021-05-28:6.475.0,putExpDateMap.2021-05-28:6.480.0,putExpDateMap.2021-05-28:6.485.0,putExpDateMap.2021-05-28:6.490.0,putExpDateMap.2021-05-28:6.495.0,putExpDateMap.2021-05-28:6.500.0,putExpDateMap.2021-05-28:6.510.0,putExpDateMap.2021-05-28:6.520.0,putExpDateMap.2021-05-28:6.530.0,putExpDateMap.2021-05-28:6.540.0
0,SPY,SUCCESS,None,SINGLE,0.0,True,False,0.1,415.43,29.0,...,"[{'putCall': 'PUT', 'symbol': 'SPY_052821P475'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P480'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P485'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P490'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P495'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P500'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P510'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P520'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P530'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P540'..."


In [8]:
explode = []

for i in initial.columns:
    try: 
        explode.append(pd.json_normalize(initial[i].explode()))
    except:
        pass

In [64]:
complete = pd.concat(explode).reset_index()
complete["delta"] = complete["delta"].astype(float)

In [194]:
delta = 0.06
dte = 4

idx = complete[complete["daysToExpiration"] == dte]["delta"].add(delta).abs().idxmin()

In [78]:
complete.columns

Index(['index', 'putCall', 'symbol', 'description', 'exchangeName', 'bid',
       'ask', 'last', 'mark', 'bidSize', 'askSize', 'bidAskSize', 'lastSize',
       'highPrice', 'lowPrice', 'openPrice', 'closePrice', 'totalVolume',
       'tradeDate', 'tradeTimeInLong', 'quoteTimeInLong', 'netChange',
       'volatility', 'delta', 'gamma', 'theta', 'vega', 'rho', 'openInterest',
       'timeValue', 'theoreticalOptionValue', 'theoreticalVolatility',
       'optionDeliverablesList', 'strikePrice', 'expirationDate',
       'daysToExpiration', 'expirationType', 'lastTradingDay', 'multiplier',
       'settlementType', 'deliverableNote', 'isIndexOption', 'percentChange',
       'markChange', 'markPercentChange', 'nonStandard', 'inTheMoney', 'mini'],
      dtype='object')

In [90]:
short_strike = dict(
    symbol=complete.iloc[[idx]]["symbol"],
    strike=complete.iloc[[idx]]["strikePrice"],
    delta=complete.iloc[[idx]]["delta"],
    bid=complete.iloc[[idx]]["bid"],
    expd=complete.iloc[[idx]]["expirationDate"],
)


In [151]:
ls_strike = str(int((short_strike["strike"]-10)))
ls_symbol = OptionSymbol('SPY', datetime.datetime.fromtimestamp(float(short_strike["expd"])/1000, tz=datetime.timezone(datetime.timedelta(0))), 'P', ls_strike).build()
ls_ask = c.get_quote(ls_symbol).json()[ls_symbol]['askPrice']
ls_delta = c.get_quote(ls_symbol).json()[ls_symbol]['delta']

In [152]:
long_strike = dict(
    symbol=ls_symbol, 
    strike=ls_strike,
    ask = ls_ask,
    delta= ls_delta,
)

In [32]:
from tda.orders.options import OptionSymbol
import tda

symbol_long = OptionSymbol('SPY', datetime.datetime(year=2021, month=5, day=28), 'P', str(397-10)).build()

In [195]:
from tda.orders.options import bull_put_vertical_open

c.place_order(220535376, bull_put_vertical_open(str(long_strike["symbol"]), short_strike["symbol"].iloc[0], 1, round(float(short_strike["bid"]-long_strike["ask"]),2)).build())

<Response [201 ]>

In [30]:
import tda

account_json = c.get_account('220535376', fields=tda.client.Client.Account.Fields.POSITIONS).json()

In [40]:
account_posits = pd.DataFrame.from_dict(account_json["securitiesAccount"]["positions"])["instrument"]

In [44]:
for i,d in enumerate(account_posits):
    pd.json_normalize(d)

In [47]:
pd.json_normalize(account_posits[3])

,assetType,cusip,symbol,description,putCall,underlyingSymbol
0,OPTION,0AMD..FI10082500,AMD_061821C82.5,AMD Jun 18 2021 82.5 Call,CALL,AMD
